In [11]:
%matplotlib widget
import numpy as np
import matplotlib.pyplot as plt
from ipywidgets import interact
import copy
from matplotlib.animation import FuncAnimation
import random

class Lattice:
    def __init__(self, limits: tuple):
        self.xlim: tuple = limits[0]
        self.ylim: tuple = limits[1]
        self.X, self.Y = np.meshgrid(np.linspace(self.xlim[0],self.xlim[1],100), np.linspace(self.ylim[0],self.ylim[1], 100))
        
    def generate_atoms(self, static: bool):
        if static:
            mosaic_layout = '''
            ABC
            '''
            layout_settings = ['A', 'B', 'C']
            density_settings = ['Low', 'Medium', 'High']
            self.fig, self.ax = plt.subplot_mosaic(mosaic_layout, layout='constrained')
            def density_setter(p,i):
                self.density = p
                list_of_centres = []
                for x in np.linspace(self.xlim[0],self.xlim[1], 2*self.density):
                    for y in np.linspace(self.ylim[0],self.ylim[1], 2*self.density):
                        list_of_centres.append((x,y))
                list_of_centres = list(set(list_of_centres))
                self.atom_list = [Atom(centre, 2,self.X, self.Y) for centre in list_of_centres]
                self.ax[layout_settings[i]].set_title(f'{density_settings[i]} Density')
                self.ax[layout_settings[i]].axis([self.xlim[0], self.xlim[1], self.ylim[0], self.ylim[1]])
                self.ax[layout_settings[i]].set_aspect('equal')
                self.ax[layout_settings[i]].set_xticks([])
                self.ax[layout_settings[i]].set_yticks([])
                for atom in self.atom_list:
                    atom.plot_wave_function(self.ax[layout_settings[i]])
                plt.show()
            values = [2,5,8]
            for i,v in enumerate(values):
                density_setter(v, i)
            
        else:
            @interact(p=(1,10))
            def density_setter(p):
                self.density = p
                list_of_centres = []
                for x in np.linspace(self.xlim[0],self.xlim[1], 2*self.density):
                    for y in np.linspace(self.ylim[0],self.ylim[1], 2*self.density):
                        list_of_centres.append((x,y))
                list_of_centres = list(set(list_of_centres))
                self.atom_list = [Atom(centre, 2,self.X, self.Y) for centre in list_of_centres]
                self.fig, self.ax = plt.subplots()
                self.fig.suptitle('Wave functions of atoms in a lattice')
                self.ax.axis([self.xlim[0], self.xlim[1], self.ylim[0], self.ylim[1]])
                self.ax.set_aspect('equal')
                for atom in self.atom_list:
                    atom.plot_wave_function(self.ax)
                plt.show()
class Atom:
    def __init__(self, centre, radius,X,Y):
        self.X, self.Y = X, Y
        self.centre = centre
        self.original_centre = copy.deepcopy(centre)
        self.radius = radius
        self.oscillation_range = (list(np.linspace(self.centre[0]-0.5, self.centre[0]+0.5, 10)), list(np.linspace(self.centre[1]-0.5, self.centre[1]+0.5, 10)))
        
    def plot_wave_function(self,ax):
        self.ax = ax
        self.orbit = ax.contour(self.X,self.Y, (lambda x,y: (x-self.centre[0])**2 + (y-self.centre[1])**2 - self.radius**2)(self.X,self.Y), [0])
        #ax.title.set_text('Wave functions of atoms in a lattice')

        
    def update(self):
        self.centre = (self.original_centre[0] + random.choice(self.oscillation_range[0]), self.original_centre[1] + random.choice(self.oscillation_range[1]))
        self.plot_wave_function(self.ax)
        return self.orbit
       
L = Lattice(((-20,20),(-20,20)))
L.generate_atoms(False) 

interactive(children=(IntSlider(value=5, description='p', max=10, min=1), Output()), _dom_classes=('widget-int…